In [1]:
!pip install yfinance
!pip install openpyxl

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 63 kB 451 kB/s            
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=4d1fc0c46eb17ac9603f5861279d56d1ae6b3a69d59f4820281f44d4ae371263
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
beatrix-jupyterlab 3.1.4 requires google-cloud-bigquery-storage, which is not installed.
gcsfs 2021.11.0 requires fsspec==2021.11.0, but you have fsspec 2021.11.1 which is incom

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/project-1-alpha/15min_OHLC_Stocks (1) (2) (1) (1).xlsx


In [3]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 99694 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for libc-bin (2.31-0ubuntu9.2) ...
     |████████████████████████████████| 269 kB 608 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for ta-lib: filename=TA_Lib-0.4.23-cp37-cp37m-linux_x86_64.whl size=2107296 sha256=54d6d4eb2448a6caa3e9902a32d67f9e0062282abf6fd0d1a5c52d577a215cf3
  Stored in directory: /root/.cache/pip/wheels/bd/91/fd/d4a665996d266298b0f90306a4e5c9000ccc5bf157539646ea
Successfully built ta-lib


In [4]:
import yfinance as yf
import talib
import pandas as pd
import copy
import numpy as np
import datetime
import plotly.express as px
import matplotlib.pyplot as plt
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [5]:
df = pd.read_excel('../input/project-1-alpha/15min_OHLC_Stocks (1) (2) (1) (1).xlsx')
df.head()

,Ticker,Date/Time,Open,High,Low,Close,Volume
0,ADANIPORTS-I,2020-06-08 09:29:59,340.75,347.15,340.75,345.75,547500
1,ADANIPORTS-I,2020-06-08 09:44:59,345.60,346.05,344.00,345.20,442500
2,ADANIPORTS-I,2020-06-08 09:59:59,345.20,345.35,342.20,342.35,215000
3,ADANIPORTS-I,2020-06-08 10:14:59,342.45,343.45,342.40,343.30,130000
4,ADANIPORTS-I,2020-06-08 10:29:59,343.30,343.30,342.00,342.65,162500


In [6]:
entry_bar1 = datetime.datetime.strptime('1000-01-01 9:30:0','%Y-%m-%d %H:%M:%S')
entry_bar2 = datetime.datetime.strptime('1000-01-01 14:45:0','%Y-%m-%d %H:%M:%S')
exit_bar = datetime.datetime.strptime('1000-01-01 15:15:0','%Y-%m-%d %H:%M:%S')
df["Date/Time"] = pd.to_datetime(df["Date/Time"])

In [7]:
df.head()

,Ticker,Date/Time,Open,High,Low,Close,Volume
0,ADANIPORTS-I,2020-06-08 09:29:59,340.75,347.15,340.75,345.75,547500
1,ADANIPORTS-I,2020-06-08 09:44:59,345.60,346.05,344.00,345.20,442500
2,ADANIPORTS-I,2020-06-08 09:59:59,345.20,345.35,342.20,342.35,215000
3,ADANIPORTS-I,2020-06-08 10:14:59,342.45,343.45,342.40,343.30,130000
4,ADANIPORTS-I,2020-06-08 10:29:59,343.30,343.30,342.00,342.65,162500


In [47]:
# initial values 
ema_1_start = 10
ema_1_end = 20+1
ema_1_step = 10
ema_2_start = 50
ema_2_end = 80+1
ema_2_step = 10
r1_start = 7
r1_end = 14+1
r1_step = 7
sl_start = 0.01
sl_end = 0.021
sl_step = 0.0025
tgt_start = 0.01
tgt_end = 0.031
tgt_step = 0.0025
tsl_start = 0.01
tsl_end = 0.021
tsl_step = 0.0025

In [48]:
result=[]
strategy = {"Ema1": None, "Ema2": None, "R1": None, "SL": None, "TGT": None, "TSL": None}

In [49]:
given = pd.read_excel('../input/project-1-alpha/15min_OHLC_Stocks (1) (2) (1) (1).xlsx',sheet_name='Sheet1')
df=given

In [50]:
def Symbols_backtesting():
    for ema1 in range(ema_1_start,ema_1_end,ema_1_step):
        for ema2 in range(ema_2_start,ema_2_end,ema_2_step):
            for r1 in range(r1_start,r1_end,r1_step):
                 for sl in np.arange(sl_start,sl_end,sl_step):
                    for tgt in np.arange(tgt_start,tgt_end,tgt_step):
                         for tsl in np.arange(tsl_start,tsl_end,tsl_step):
                            strategy = {"Ema1": None, "Ema2": None, "R1": None, "SL": None, "TGT": None, "TSL": None}
                            strategy["Ema1"]=ema1
                            strategy["Ema2"]=ema2
                            strategy["R1"]=r1
                            strategy["SL"]=sl
                            strategy["TGT"]=tgt
                            strategy["TSL"]=tsl
                            all_trades = []
                            df["MA_1"] = talib.MA(df["Close"], timeperiod=ema1)
                            df["MA_2"] = talib.MA(df["Close"], timeperiod=ema2)
                            df["RSI_14"] = talib.RSI(df["Close"], timeperiod=r1)
                            trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                            initial_date = datetime.datetime(1,1,1)
                            position = None

                            for i in df.index[49:]:
                                if position=="Buy" and trade["Symbol"] is not None:
                                    if df["Close"][i]<=trade["Entry"]-(trade["Entry"]*sl):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="SL Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None
                                    elif df["Close"][i]>=trade["Entry"]+(trade["Entry"]*tgt):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="TGT Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None     
                                    elif df["Close"][i]<=trade["Entry"]-(df["Close"][i-1]*tsl):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="TSL Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None     
                                elif position=="Sell" and trade["Symbol"] is not None:
                                    if df["Close"][i]>=trade["Entry"]+(trade["Entry"]*sl):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="SL Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None
                                    elif df["Close"][i]<=trade["Entry"]-(trade["Entry"]*tgt):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="TGT Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None     
                                    elif df["Close"][i]>=trade["Entry"]+(df["Close"][i-1]*tsl):
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="TSL Trigger"
                                        all_trades.append(copy.deepcopy(trade))
                                        # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None     
                                if df["MA_1"][i] > df["MA_2"][i] and df["RSI_14"][i] > 60 and position != "Buy":
                                    if trade["Symbol"] is not None:
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="Long"
                                        all_trades.append(copy.deepcopy(trade))
                                    if position is not None and (df["Date/Time"][i].time() > entry_bar1.time()) and df["Date/Time"][i].time() < entry_bar2.time():
                                        trade["Symbol"] = "ADANIPORTS-I"
                                        trade["Buy/Sell"] = "Buy"
                                        trade["Entry"] = df["Close"][i]
                                        trade["Entry Date"] = df["Date/Time"][i]
                                    position = "Buy"
                                if df["MA_1"][i] < df["MA_2"][i] and df["RSI_14"][i] < 40 and position != "Sell":
                                    if trade["Symbol"] is not None:
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="Short"
                                        all_trades.append(copy.deepcopy(trade))
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                    if position is not None and (df["Date/Time"][i].time() > entry_bar1.time()) and df["Date/Time"][i].time() < entry_bar2.time():     
                                        trade["Symbol"] = "ADANIPORTS-I"
                                        trade["Buy/Sell"] = "Sell"
                                        trade["Entry"] = df["Close"][i]
                                        trade["Entry Date"] = df["Date/Time"][i]
                                    position = "Sell"
                                if df["Date/Time"][i].time()>exit_bar.time() and position is not None:
                                    if trade["Symbol"] is not None:
                                        trade["Exit"] = df["Close"][i]
                                        trade["Exit Date"] = df["Date/Time"][i]
                                        trade["Detail"]="Time Out"
                                        all_trades.append(copy.deepcopy(trade))
                                        trade = {"Symbol": None,"Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                         # trade ko none karna hei ya nhi aur position ? 
                                        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                                        position=None
                            data1=pd.DataFrame(data=all_trades)
                            all_trades=[]
                            if data1.empty == False:
                                d = pd.DataFrame(data1)
                                d["P/L"] = np.where(d["Buy/Sell"] == "Buy", d["Exit"] - d["Entry"],d["Entry"] - d["Exit"])
                                d["P/L"] = d["Exit"] - d["Entry"]
                                d["Return"] = d["P/L"].cumsum()
                                d["Drawdown"] = d["Return"] - (d["Return"].cummax().apply(lambda x: x if x > 0 else 0))
                                df_final=d.tail(1)
                                win=0
                                loss=0
                                tradecount=0
                                for x in d.index[:]:
                                    if d["P/L"][x]>=0:
                                        win=win+1
                                    else:
                                        loss=loss+1
                                    tradecount=tradecount+1
                                count = df.groupby(df['Date/Time'].dt.date).size()
                                strategy["Days"]=count.shape[0]
                                
                                u=0
                                for x in d.index[:]:
                                    u=u+d["P/L"][x]
                                strategy["PL"] = u
                                strategy["Trade"]=tradecount
                                strategy["W"] = win
                                strategy["L"] = loss
                                strategy["PL/Trade"]=u/tradecount
                                result.append(copy.deepcopy(strategy))
                            strategy = {"Ema1": None, "Ema2": None, "R1": None, "SL": None, "TGT": None, "TSL": None, "Return":None}
        
Symbols_backtesting()
df1 = pd.DataFrame(data=result)
result=[]
df1.head()

,Ema1,Ema2,R1,SL,TGT,TSL,Days,PL,Trade,W,L,PL/Trade
0,10,50,7,0.01,0.01,0.0100,228,162.9,67,44,23,2.431343
1,10,50,7,0.01,0.01,0.0125,228,162.9,67,44,23,2.431343
2,10,50,7,0.01,0.01,0.0150,228,162.9,67,44,23,2.431343
3,10,50,7,0.01,0.01,0.0175,228,162.9,67,44,23,2.431343
4,10,50,7,0.01,0.01,0.0200,228,162.9,67,44,23,2.431343


In [52]:
df1.to_csv('Sheet_1_Detail.csv',index=False)
df1=df1.sort_values(by="PL", ascending=False)
df1.head(1)

,Ema1,Ema2,R1,SL,TGT,TSL,Days,PL,Trade,W,L,PL/Trade
182,10,50,7,0.02,0.01,0.015,228,183.45,68,45,23,2.697794


**Above shows the best combination of parameters for maximum PL**

**Now we use this combination to show the trades in this**

In [53]:
# Model for manul value input
def Symbols_backtesting():
    ema1=10
    ema2=50
    r1=7
    sl=0.02
    tgt=0.01	
    tsl=0.015
    all_trades = []
    df["MA_10"] = talib.MA(df["Close"], timeperiod=ema1)
    df["MA_50"] = talib.MA(df["Close"], timeperiod=ema2)
    df["RSI_14"] = talib.RSI(df["Close"], timeperiod=r1)
    trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
    initial_date = datetime.datetime(1,1,1)
    position = None
    for i in df.index[49:]:
        if position=="Buy" and trade["Symbol"] is not None:
            if df["Close"][i]<=trade["Entry"]-(trade["Entry"]*sl):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="SL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
            elif df["Close"][i]>=trade["Entry"]+(trade["Entry"]*tgt):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="TGT Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
            elif df["Close"][i]<=trade["Entry"]-(df["Close"][i-1]*tsl):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="TSL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
        elif position=="Sell" and trade["Symbol"] is not None:
            if df["Close"][i]>=trade["Entry"]+(trade["Entry"]*sl):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="SL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
            elif df["Close"][i]<=trade["Entry"]-(trade["Entry"]*tgt):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="TGT Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
            elif df["Close"][i]>=trade["Entry"]+(df["Close"][i-1]*tsl):
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="TSL Trigger"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
        if df["MA_10"][i] > df["MA_50"][i] and df["RSI_14"][i] > 60 and position != "Buy":
            if trade["Symbol"] is not None:
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="Sell_Exit Buy"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
            if position is not None and (df["Date/Time"][i].time() > entry_bar1.time()) and df["Date/Time"][i].time() < entry_bar2.time():
                trade["Symbol"] = "ADANIPORTS-I"
                trade["Buy/Sell"] = "Buy"
                trade["Entry"] = df["Close"][i]
                trade["Entry Date"] = df["Date/Time"][i]
            position = "Buy"
        if df["MA_10"][i] < df["MA_50"][i] and df["RSI_14"][i] < 40 and position != "Sell":
            if trade["Symbol"] is not None:
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="Buy_Exit Sell"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
            if position is not None and (df["Date/Time"][i].time() > entry_bar1.time()) and df["Date/Time"][i].time() < entry_bar2.time():     
                trade["Symbol"] = "ADANIPORTS-I"
                trade["Buy/Sell"] = "Sell"
                trade["Entry"] = df["Close"][i]
                trade["Entry Date"] = df["Date/Time"][i]
            position = "Sell"
        if df["Date/Time"][i].time()>exit_bar.time() and position is not None:
            if trade["Symbol"] is not None:
                trade["Exit"] = df["Close"][i]
                trade["Exit Date"] = df["Date/Time"][i]
                trade["Detail"]="Time Out"
                all_trades.append(copy.deepcopy(trade))
                trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None, "Detail":None}
                position=None
    return all_trades


data = Symbols_backtesting()
Trade_details = pd.DataFrame(data=data)
Trade_details.head(20)
# print(Trade_details.shape)

,Symbol,Buy/Sell,Entry,Entry Date,Exit,Exit Date,Detail
0,ADANIPORTS-I,Sell,343.30,2020-06-10 10:29:59,347.25,2020-06-10 15:14:59,Sell_Exit Buy
1,ADANIPORTS-I,Sell,342.40,2020-06-11 11:14:59,337.65,2020-06-11 14:44:59,TGT Trigger
2,ADANIPORTS-I,Buy,343.10,2020-06-12 14:44:59,347.00,2020-06-12 15:29:59,TGT Trigger
3,ADANIPORTS-I,Sell,339.30,2020-06-16 13:29:59,341.20,2020-06-16 15:29:59,Time Out
4,ADANIPORTS-I,Sell,337.25,2020-06-18 09:59:59,343.35,2020-06-18 13:44:59,TSL Trigger
5,ADANIPORTS-I,Sell,350.00,2020-06-23 09:44:59,353.00,2020-06-23 11:59:59,Sell_Exit Buy
6,ADANIPORTS-I,Buy,353.00,2020-06-23 11:59:59,355.05,2020-06-23 15:29:59,Time Out
7,ADANIPORTS-I,Buy,345.55,2020-06-30 09:59:59,345.00,2020-06-30 15:29:59,Time Out
8,ADANIPORTS-I,Sell,358.40,2020-07-06 14:44:59,362.90,2020-07-06 15:29:59,Time Out
9,ADANIPORTS-I,Buy,315.85,2020-07-20 10:14:59,313.30,2020-07-20 12:29:59,Buy_Exit Sell


In [26]:
strategy = {"Ema1": None, "Ema2": None, "R1": None, "SL": None, "TGT": None, "TSL": None, "Return":None}        
result=[]
data1=Trade_details
all_trades=[]
if data1.empty == False:
    d = pd.DataFrame(data1)
    d["P/L"] = np.where(d["Buy/Sell"] == "Buy", d["Exit"] - d["Entry"],d["Entry"] - d["Exit"])
    d["P/L"] = d["Exit"] - d["Entry"]
    d["Return"] = d["P/L"].cumsum()
    d["Drawdown"] = d["Return"] - (d["Return"].cummax().apply(lambda x: x if x > 0 else 0))
    df_final=d.tail(1)
    win=0
    loss=0
    tradecount=0
    for x in d.index[:]:
        if d["P/L"][x]>=0:
            win=win+1
        else:
            loss=loss+1
        tradecount=tradecount+1
    count = df.groupby(df['Date/Time'].dt.date).size()
    strategy["Days"]=count.shape[0]

    u=0
    for x in d.index[:]:
        u=u+d["P/L"][x]
    strategy["PL"] = u
    strategy["Trade"]=tradecount
    strategy["W"] = win
    strategy["L"] = loss
    strategy["PL/Trade"]=u/tradecount
    result.append(copy.deepcopy(strategy))
strategy = {"Ema1": None, "Ema2": None, "R1": None, "SL": None, "TGT": None, "TSL": None, "Return":None}        
df1 = pd.DataFrame(data=result)
result=[]
df1.head()

,Ema1,Ema2,R1,SL,TGT,TSL,Return,Days,PL,Trade,W,L,PL/Trade
0,None,None,None,None,None,None,None,228,178.6,67,44,23,2.665672


In [55]:
Trade_details.to_csv('Trade_Detail_1.csv',index=False)

**Lets plot the signals**

In [ ]:
import plotly.graph_objects as go
df1=Trade_details
fig = go.Figure()
# Add traces
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry"],
                    mode='lines+markers',
                    name='Buy_Entry'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit"],
                    mode='lines+markers',
                    name='Buy_Exit'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    # mode='lines',
                    # name='lines'))

fig.show()

In [ ]:
fig = go.Figure()
# Add traces

fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry"],
                    mode='markers',
                    name='Buy_Entry'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Sell"]["Entry Date"], y=df1.loc[df1["Buy/Sell"]=="Sell"]["Entry"],
                    mode='markers',
                    name='Sell_Entry'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit"],
                    mode='markers',
                    name='Buy_Exit'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Sell"]["Exit Date"], y=df1.loc[df1["Buy/Sell"]=="Sell"]["Exit"],
                    mode='markers',
                    name='Sell_Exit'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    # mode='lines',
                    # name='lines'))

fig.show()

In [ ]:
fig = go.Figure()
# Add traces
fig.add_trace(go.Line(x=df["Date/Time"], y=df["Close"],
                    mode='lines',
                    name='Price'))
fig.add_trace(go.Line(x=df["Date/Time"], y=df["MA_10"],
                    mode='lines',
                    name='MA_10'))
fig.add_trace(go.Line(x=df["Date/Time"], y=df["MA_50"],
                    mode='lines',
                    name='MA_50'))
fig.add_trace(go.Scatter(x=df1["Entry Date"], y=df1["Entry"],
                    mode='markers',
                    name='Entry'))
fig.add_trace(go.Scatter(x=df1["Exit Date"], y=df1["Exit"],
                    mode='markers',
                    name='Exit'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    # mode='lines',
                    # name='lines'))

fig.show()

In [ ]:
fig = go.Figure()
# Add traces
fig.add_trace(go.Line(x=df["Date/Time"], y=df["Close"],
                    mode='lines',
                    name='Price'))
fig.add_trace(go.Line(x=df["Date/Time"], y=df["MA_10"],
                    mode='lines',
                    name='MA_10'))
fig.add_trace(go.Line(x=df["Date/Time"], y=df["MA_50"],
                    mode='lines',
                    name='MA_50'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Entry"],
                    mode='markers',
                    name='Buy_Entry'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Sell"]["Entry Date"], y=df1.loc[df1["Buy/Sell"]=="Sell"]["Entry"],
                    mode='markers',
                    name='Sell_Entry'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit Date"], y=df1.loc[df1["Buy/Sell"]=="Buy"]["Exit"],
                    mode='markers',
                    name='Buy_Exit'))
fig.add_trace(go.Scatter(x=df1.loc[df1["Buy/Sell"]=="Sell"]["Exit Date"], y=df1.loc[df1["Buy/Sell"]=="Sell"]["Exit"],
                    mode='markers',
                    name='Sell_Exit'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    # mode='lines',
                    # name='lines'))

fig.show()